In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
pd.options.display.float_format = '{:.4f}'.format
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
pd.set_option("display.max_rows", None, "display.max_columns", None)
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from statsmodels.formula.api import ols

In [2]:
cck_df = pd.read_csv('cck-geo.csv')

In [3]:
cck_df = cck_df[(cck_df['month'] >= '2015-01') & (cck_df['month'] <= '2020-08')]

In [4]:
len(cck_df)

5025

In [5]:
cck_amenities_geo_df = pd.read_csv('cck_amenities_geocode.csv')

In [6]:
# Create a new column 'Distance' and measure the distance between every possible combination of stations

from math import *

# Function to calculate distance between latitude-longitude pairs. Source of formula: http://andrew.hedges.name/experiments/haversine/

def distance(lat1, long1, lat2, long2):
    
    lat1 = radians(lat1)
    long1 = radians(long1)
    lat2 = radians(lat2)
    long2 = radians(long2)
    R = 6373
    
    dlong = long2 - long1
    dlat = lat2 - lat1
    a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlong/2))**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    
    return distance

In [7]:
cck_df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,complete_address,Lat,Long
28285,2015-01,CHOA CHU KANG,3 ROOM,5,TECK WHYE AVE,01 TO 03,79,Model A,1984,295000.0000,68,5 TECK WHYE AVE,1.3775,103.7562
28286,2015-01,CHOA CHU KANG,4 ROOM,15,TECK WHYE LANE,01 TO 03,92,New Generation,1979,320000.0000,63,15 TECK WHYE LANE,1.3795,103.7551
28287,2015-01,CHOA CHU KANG,4 ROOM,459,CHOA CHU KANG AVE 4,07 TO 09,103,Premium Apartment,2000,323888.0000,84,459 CHOA CHU KANG AVE 4,1.3782,103.7351
28288,2015-01,CHOA CHU KANG,4 ROOM,206,CHOA CHU KANG CTRL,01 TO 03,104,Model A,1989,325000.0000,73,206 CHOA CHU KANG CENTRAL,1.3826,103.7445
28289,2015-01,CHOA CHU KANG,4 ROOM,690A,CHOA CHU KANG CRES,04 TO 06,90,Model A,2003,330000.0000,87,690A CHOA CHU KANG CRES,1.3995,103.7507


In [8]:
cck_df['dist_nearest_train'] = None
cck_df['dist_nearest_prisch'] = None
cck_df['dist_nearest_secsch'] = None
cck_df['dist_nearest_mall'] = None

In [9]:
cck_df.dtypes

month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm           int64
flat_model              object
lease_commence_date      int64
resale_price           float64
remaining_lease         object
complete_address        object
Lat                    float64
Long                   float64
dist_nearest_train      object
dist_nearest_prisch     object
dist_nearest_secsch     object
dist_nearest_mall       object
dtype: object

In [10]:
# Getting nearest distance to mall

for i in range(len(cck_df)):
    
    dist_mall_list = []

    for j in range(4):

        dist_mall = distance(cck_df.iloc[i, 12], cck_df.iloc[i, 13], cck_amenities_geo_df.iloc[j, -2], cck_amenities_geo_df.iloc[j, -1])

        dist_mall_list.append(dist_mall)

    cck_df.iloc[i, -1] = min(dist_mall_list)

In [11]:
# Getting nearest distance to primary school

pri_sch_index = [4, 6, 7, 8, 10, 11, 12, 16]

for i in range(len(cck_df)):
    
    dist_prisch_list = []
    
    for j in pri_sch_index:

        dist_prisch = distance(cck_df.iloc[i, 12], cck_df.iloc[i, 13], cck_amenities_geo_df.iloc[j, -2], cck_amenities_geo_df.iloc[j, -1])

        dist_prisch_list.append(dist_prisch)

    cck_df.iloc[i, -3] = min(dist_prisch_list)

In [12]:
# Getting nearest distance to secondary school

sec_sch_index = [5, 13, 14, 15, 17, 18]

for i in range(len(cck_df)):
    
    dist_secsch_list = []
    
    for j in sec_sch_index:

        dist_secsch = distance(cck_df.iloc[i, 12], cck_df.iloc[i, 13], cck_amenities_geo_df.iloc[j, -2], cck_amenities_geo_df.iloc[j, -1])

        dist_secsch_list.append(dist_secsch)

    cck_df.iloc[i, -2] = min(dist_secsch_list)

In [13]:
# Getting nearest distance to train station

for i in range(len(cck_df)):
    
    dist_train_list = []
    
    for j in range(-6,0,1):

        dist_train = distance(cck_df.iloc[i, 12], cck_df.iloc[i, 13], cck_amenities_geo_df.iloc[j, -2], cck_amenities_geo_df.iloc[j, -1])

        dist_train_list.append(dist_train)

    cck_df.iloc[i, -4] = min(dist_train_list)

In [14]:
cck_df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,complete_address,Lat,Long,dist_nearest_train,dist_nearest_prisch,dist_nearest_secsch,dist_nearest_mall
28285,2015-01,CHOA CHU KANG,3 ROOM,5,TECK WHYE AVE,01 TO 03,79,Model A,1984,295000.0000,68,5 TECK WHYE AVE,1.3775,103.7562,0.3571,0.7664,0.6347,0.4955
28286,2015-01,CHOA CHU KANG,4 ROOM,15,TECK WHYE LANE,01 TO 03,92,New Generation,1979,320000.0000,63,15 TECK WHYE LANE,1.3795,103.7551,0.4611,0.5137,0.4015,0.2992
28287,2015-01,CHOA CHU KANG,4 ROOM,459,CHOA CHU KANG AVE 4,07 TO 09,103,Premium Apartment,2000,323888.0000,84,459 CHOA CHU KANG AVE 4,1.3782,103.7351,1.2652,0.2622,0.5610,0.3932
28288,2015-01,CHOA CHU KANG,4 ROOM,206,CHOA CHU KANG CTRL,01 TO 03,104,Model A,1989,325000.0000,73,206 CHOA CHU KANG CENTRAL,1.3826,103.7445,0.2375,0.0000,0.6905,0.2777
28289,2015-01,CHOA CHU KANG,4 ROOM,690A,CHOA CHU KANG CRES,04 TO 06,90,Model A,2003,330000.0000,87,690A CHOA CHU KANG CRES,1.3995,103.7507,0.4316,0.2915,0.5170,0.5305


In [15]:
# change flat_type column
flat_type_dic = {'2 ROOM': 2, '3 ROOM':3, '4 ROOM': 4, '5 ROOM' : 5, 'EXECUTIVE' : 6}
cck_df['flat_type'] = cck_df['flat_type'].map(flat_type_dic)

In [16]:
# changing storey_range
storey_type_dic = {'01 TO 03':'storey_1_9','04 TO 06':'storey_1_9','07 TO 09':'storey_1_9', '10 TO 12':'storey_10_18'
                   , '13 TO 15':'storey_10_18', '16 TO 18':'storey_10_18', '19 TO 21':'storey_19_27','22 TO 24':'storey_19_27',
                   '25 TO 27':'storey_19_27'}
cck_df['storey_range'] = cck_df['storey_range'].map(storey_type_dic)

In [17]:
cck_df = pd.concat((cck_df,pd.get_dummies(cck_df.loc[:, 'storey_range'])),axis=1)

In [18]:
# Getting price per sq metre

cck_df['price_psm'] = cck_df['resale_price'] / cck_df['floor_area_sqm']

In [19]:
cck_df.dtypes

month                   object
town                    object
flat_type                int64
block                   object
street_name             object
storey_range            object
floor_area_sqm           int64
flat_model              object
lease_commence_date      int64
resale_price           float64
remaining_lease         object
complete_address        object
Lat                    float64
Long                   float64
dist_nearest_train      object
dist_nearest_prisch     object
dist_nearest_secsch     object
dist_nearest_mall       object
storey_10_18             uint8
storey_19_27             uint8
storey_1_9               uint8
price_psm              float64
dtype: object

In [20]:
cck_df['num_train'] = None
cck_df['num_prisch'] = None
cck_df['num_secsch'] = None
cck_df['num_mall'] = None

In [21]:
# Getting the number of malls within 1km

for i in range(len(cck_df)):
    
    num_mall = 0
    
    for j in range(4):

        dist_mall = distance(cck_df.iloc[i, 12], cck_df.iloc[i, 13], cck_amenities_geo_df.iloc[j, -2], cck_amenities_geo_df.iloc[j, -1])
        
        if dist_mall <= 1:
            
            num_mall += 1
            
        else:
            
            continue
    
    cck_df.iloc[i, -1] = num_mall

In [22]:
# Getting the number of primary schools within 1km

for i in range(len(cck_df)):
    
    num_prisch = 0
    
    for j in pri_sch_index:

        dist_prisch = distance(cck_df.iloc[i, 12], cck_df.iloc[i, 13], cck_amenities_geo_df.iloc[j, -2], cck_amenities_geo_df.iloc[j, -1])
        
        if dist_prisch <= 1:
            
            num_prisch += 1
            
        else:
            
            continue

    cck_df.iloc[i, -3] = num_prisch

In [23]:
# Getting the number of secondary schools within 1km

for i in range(len(cck_df)):
    
    num_secsch = 0
    
    for j in sec_sch_index:

        dist_secsch = distance(cck_df.iloc[i, 12], cck_df.iloc[i, 13], cck_amenities_geo_df.iloc[j, -2], cck_amenities_geo_df.iloc[j, -1])
        
        if dist_secsch <= 1:
            
            num_secsch += 1
            
        else:
            
            continue

    cck_df.iloc[i, -2] = num_secsch

In [24]:
# Getting the number of train stations within 1km

for i in range(len(cck_df)):
    
    num_train = 0
    
    for j in range(-6,0,1):

        dist_train = distance(cck_df.iloc[i, 12], cck_df.iloc[i, 13], cck_amenities_geo_df.iloc[j, -2], cck_amenities_geo_df.iloc[j, -1])
        
        if dist_train <= 1:
            
            num_train += 1
            
        else:
            
            continue

    cck_df.iloc[i, -4] = num_train

In [25]:
cck_df['train_station_ismrt'] = None

In [26]:
# Finding out if the nearest train station is MRT/LRT

for i in range(len(cck_df)):
    
    min_dist_train = 10
    
    index = -6
    
    for j in range(-6,0,1):

        dist_train = distance(cck_df.iloc[i, 12], cck_df.iloc[i, 13], cck_amenities_geo_df.iloc[j, -2], cck_amenities_geo_df.iloc[j, -1])
        
        if dist_train < min_dist_train:
            
            min_dist_train = dist_train
            
            index = j
            
        else: 
            
            continue
    
    station_type = cck_amenities_geo_df.iloc[index, 3]
    
    if station_type == "lrt":
        
        cck_df.iloc[i, -1] = 0
        
    elif station_type == "mrt":
        
        cck_df.iloc[i, -1] = 1
        
    else:
        
        cck_df.iloc[i, -1] = None

In [27]:
remaininglease_df = pd.read_csv('cck_df_remaininglease.csv')

In [28]:
cck_df.loc[:,'remaining_lease'] = remaininglease_df.loc[:,'remaining_lease'].values

In [29]:
cck_df.to_csv('regression.csv')